<a href="https://colab.research.google.com/github/delhian/NIR/blob/main/NIR/%D0%9D%D0%98%D0%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install umap-learn > /dev/null

In [62]:
import re
# import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import random
import umap.umap_ as umap
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm

from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [ ]:
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# name = '/content/drive/MyDrive/Anthony & Margroff - K1 - Dragon\'s Gold.txt'
name = '/content/drive/MyDrive/text.txt'
with open(name, 'r', ) as file:
    text = file.read()
print(len(text))

541415051


In [ ]:
# sub = '[\?\.\!]([^\?\.\!]*?[^a-z-]'+'red'+'[^a-z-][^\?\.\!]*?[\?\.\!])'

# prog = re.compile(sub)

# for ind, m in enumerate(re.finditer(prog, text.lower())):
#   print('**********')
#   print(text[m.start() + 1: m.end()])
#   print(m.group(0))
#   print('**********')
#   if ind  == 5:
#     break


In [78]:
%%time

phrase_list = ['pink', 'crimson','red','blue','brown' ,'orange', 'grey','green', 'gray', 'yellow', 'white', 'black','purple']
# phrase_list = ['gray']

def f_dfmake(text, phrase_list, n_max):

  # phrase_list = ['red',]
  emb = []
  for token in phrase_list :
    # print(token)
    emb_token = []
    sub = '[\?\.\!]([^\?\.\!]*?[^a-z-]'+token+'[^a-z-][^\?\.\!]*?[\?\.\!])'
    prog = re.compile(sub)
    iter = re.finditer(prog, text.lower())
    for ind, seq in enumerate(iter):
      txt = text[seq.start() + 1: seq.end()]
      txt_lower = seq.group(0)
      # print(seq.group(0))
      seq_tokenized = tokenizer.tokenize(txt_lower)
      seq_tokenized_length = len(seq_tokenized)
      if seq_tokenized_length > 512: ## too long sequence!
        # print(f'seq_tokenized_length too long: {seq_tokenized_length} !')
        continue
      # print(seq_tokenized)
      ind = seq_tokenized.index(token)
      encoded_input = tokenizer(txt_lower, return_tensors='pt')
      out = model(**encoded_input)[0][0,1:-1]
      out = out[ind].detach().numpy()
      emb_token.append([token, txt , out])
      # print(len(emb_token))
      if len(emb_token)  == n_max:
        break
    emb += emb_token
  emb = pd.DataFrame(emb, columns = ['token', 'txt', 'word_embedding'])
  print(emb.token.value_counts(dropna=False))
  return emb

n_max = 50
emb = f_dfmake (text, phrase_list, n_max)

orange     50
purple     50
white      50
blue       50
crimson    50
green      50
brown      50
gray       50
red        50
black      50
yellow     50
grey       50
pink       50
Name: token, dtype: int64
CPU times: user 5min 37s, sys: 6.73 s, total: 5min 43s
Wall time: 3min 26s


In [59]:
def f_plot(df, n_neighbors , min_dist, phrase_list, phrase_list_extended):
    
    phases =[]
    for x in phrase_list:
        phases.append([x,[y[0] for y in phrase_list_extended if y[1] == x]])
    
    c = [c for c in df.columns if 'word_embedding' in c][0]
    arr_proj = umap.UMAP(n_neighbors = n_neighbors, min_dist =min_dist)\
    .fit_transform(np.array(df[c].values.tolist()))
    df['x'], df['y'] = arr_proj[:,0], arr_proj[:,1]

    plt.figure(figsize=(13, 10))
    for phrase in phases:
        df_phrase = df[df['token'].str.lower() == phrase[0]]
        if phrase[0] in list(matplotlib.colors.cnames.keys()):
            plt.scatter(df_phrase['x'], 
                        df_phrase['y'],
                        linewidths = 0.2, 
                        marker = "o", 
                        color=phrase[0], 
                        edgecolors='b',
                        s = 30,
                        label=phrase[0])
        else:
            plt.scatter(df_phrase['x'], 
                        df_phrase['y'],
                        linewidths = 0.2, 
                        marker = "o",  
                        edgecolors='b',
                        s = 30,
                        label=phrase[0])
        for phrase_new in phrase[1]:
            print(phrase_new)
            df_phrase = df[df['token'].str.lower() == phrase_new]
            plt.scatter(df_phrase['x'],
                        df_phrase['y'],
                        linewidths = 0.2, 
                        marker = "*",
                        color= tuple([abs(x - random.uniform(0.1, 0.25)) for x in colors.to_rgb(phrase[0])]),
                        edgecolors='b',
                        s = 50,
                        label=phrase_new)
            
            
        # plt.legend()

#     for i in range(df.shape[0]):
#         plt.text(df.iloc[i]['x'], df.iloc[i]['y'], str(i),
#                  fontsize = 'x-small')

    plt.legend(loc='upper right', 
                bbox_to_anchor=(1.13, 1), 
                markerscale=1.7,
                ncol=1, 
                fontsize = 'small')
    plt.title(name)
    plt.show()

In [80]:
emb

,token,txt,word_embedding
0,pink,"\n\tEdorion drew a deep breath, puffing his pi...","[-0.8603322, -0.38904473, 0.104271926, -0.4258..."
1,pink,There were pink ones in between to show that ...,"[-0.15525538, -0.18555067, 0.071046904, -0.084..."
2,pink,"\n""Morning, Jack,"" Cathy said, entering the ki...","[-0.26438084, -0.62798965, 0.19889872, -0.0676..."
3,pink,She was really clean for the first time in ou...,"[-0.6256314, -0.8682387, 0.35934496, -0.477517..."
4,pink,I'm in the pink. How about yourself?,"[-0.62260383, -0.5831793, -0.08096335, -0.0307..."
...,...,...,...
645,purple,There was only one thing on it apart\nfrom hi...,"[0.08743012, 0.17528136, 0.27562228, 0.1302258..."
646,purple,"\nNeville Longbottom bought a large, evil-smel...","[-0.2571932, 0.7186529, 0.27619186, -0.38802, ..."
647,purple,"He\ntried to point this out to Ron, but Ron w...","[0.08252537, -0.17007811, 0.9250924, -0.183426..."
648,purple,\n\nHarry was just mounting his broom when Pro...,"[0.12921508, 0.30126384, 0.28545302, -0.218880..."


In [89]:
def f_plot(df, n_neighbors , min_dist, phrase_list):

  arr_proj = umap.UMAP(n_neighbors = n_neighbors, min_dist =min_dist).\
  fit_transform(np.array(df['word_embedding'].values.tolist()))
  
  df['x'], df['y'] = arr_proj[:,0], arr_proj[:,1]

  p = figure(tools="pan,wheel_zoom,reset,save",
            toolbar_location="above",
            title=name)

  for phrase in phrase_list:
    df_phrase = df[df['token'].str.lower() == phrase]
    source = ColumnDataSource(data=dict(x1=arr_proj[:,0],
                                        x2=arr_proj[:,1],
                                        names=df.index))
    p.scatter(x="x1", y="x2", size=8, source=source)
    
    if phrase in list(matplotlib.colors.cnames.keys()):
      print(phrase)

      labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                        text_font_size="8pt", text_color = phrase,
                        source=source, text_align='center')
    p.add_layout(labels)
  else:
    print('phrase:', phrase)
  show(p)


  #     p.scatter(df_phrase['x'],
  #               df_phrase['y'],
  #                       linewidths = 0.2, 
  #                       marker = "o", 
  #                       color=phrase[0], 
  #                       edgecolors='b',
  #                       s = 30,
  #                       label=phrase[0])
  #       else:
  #           p.scatter(df_phrase['x'], 
  #                       df_phrase['y'],
  #                       linewidths = 0.2, 
  #                       marker = "o",  
  #                       edgecolors='b',
  #                       s = 30,
  #                       label=phrase[0])

  # p.scatter(x="x1", y="x2", size=8, source=source)

  # labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
  #                   text_font_size="8pt", text_color="#555555",
  #                   source=source, text_align='center')


In [90]:
# phrase_list_extended = []
f_plot(emb, n_neighbors =15, min_dist = 0.5, phrase_list= phrase_list, )

pink
crimson
red
blue
brown
orange
grey
green
gray
yellow
white
black
purple
phrase: purple
